In [1]:
import numpy as np
from keras.datasets import mnist
import random

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

# Flatten the images and convert labels to integers
x_train = x_train.reshape((x_train.shape[0], -1))
x_test = x_test.reshape((x_test.shape[0], -1))

y_train = np.array(y_train)
y_test = np.array(y_test)

def create_custom_nn(num_layers, layer_neurons, input_shape, output_shape, epsi):
    model = Sequential()
    model.add(Dense(layer_neurons[0], activation='relu', input_shape=(784,))) 
    for i in range(num_layers-1):
        model.add(Dense(layer_neurons[i+1], activation='relu'))
    model.add(Dense(output_shape, activation='softmax'))
    model.add(BatchNormalization(epsilon=epsi))

    return model


def test_model(chromosome):
    model = create_custom_nn(chromosome[0], chromosome[1], 784, 10, chromosome[2])
    custom_optimizer = Adam(learning_rate=(chromosome[3]/50))

    model.compile(optimizer=custom_optimizer,
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test), verbose=0)

    test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)

    return test_accuracy


In [3]:
def initialize_population(sol_per_pop, num_genes, nb_neurons_possible, low_end_randomized, high_end_randomized):
    population = []
    for _ in range(sol_per_pop):
        chromosome = []
        nb_layers = random.randint(1, 2)
        chromosome.append(nb_layers)
        nb_on_layers=[]
        for _ in range(nb_layers):
            nb_on_layers.append(random.choice(nb_neurons_possible))
        chromosome.append(nb_on_layers)
        chromosome.append(random.uniform(low_end_randomized, high_end_randomized))
        chromosome.append(random.uniform(low_end_randomized, high_end_randomized))
        population.append(chromosome)
    return population


def select_parents(population, num_parents_mating):
    accuracies = [test_model(chromosome) for chromosome in population]

    individuals_with_accuracies = list(zip(population, accuracies))

    individuals_with_accuracies.sort(key=lambda x: x[1], reverse=True)

    selected_parents = individuals_with_accuracies[:num_parents_mating]
    parents, parent_accuracies = zip(*selected_parents)
    parents = list(parents)

    return parents, parent_accuracies


def crossover(parents):
    offspring = []
    for i in range(sol_per_pop - num_parents_mating):
        parent1 = parents[0]
        parent2 = parents[1]
        child = [0, [], 0, 0]
        for j in range(len(parent1)):
            if j == 1:
                for k in range(child[0]):
                    if (parent1[0] > k) and (parent2[0] > k):
                        if random.random() < 0.5:
                            child[1].append(parent1[j][k])
                        else:
                            child[1].append(parent2[j][k])
                    elif parent1[0] > k:
                        child[1].append(parent1[j][k])
                    elif parent2[0] > k:
                        child[1].append(parent2[j][k])
                    else:
                        child[1].append(random.choice(nb_neurons_possible))
            else:
                if random.random() < 0.5:
                    child[j]=parent1[j]
                else:
                    child[j]=parent2[j]
        offspring.append(child)
    for j in range(len(offspring)):
        if isinstance(offspring[j][1], int):
            offspring[j][1] = [offspring[j][1]]
    return offspring


def mutate(offspring):
    for i in range(len(offspring)):
        for j in range(len(offspring[i])):
            if j == 0:
                if random.random() < 0.1 and offspring[i][j] == 2:
                    x = random.randint(-1, 1)
                    offspring[i][j] = offspring[i][j] + x
                    if x == -1:
                        offspring[i][j+1] = offspring[i][j+1][:-1]
                    elif x == 1:
                        offspring[i][j+1].append(random.choice(nb_neurons_possible))
                elif random.random() < 0.1 and offspring[i][j] == 1:
                    x = random.randint(0, 1)
                    offspring[i][j] = offspring[i][j] + x
                    if x == 1:
                        offspring[i][j+1].append(random.choice(nb_neurons_possible))
                elif random.random() < 0.1 and offspring[i][j] == 3:
                    x = random.randint(-1, 0)
                    offspring[i][j] = offspring[i][j] + x
                    if x == -1:
                        offspring[i][j+1] = offspring[i][j+1][:-1]
            elif j == 1:
                if random.random()<0.1 and offspring[i][0]==1:
                    offspring[i][1] = nb_neurons_possible[random.randint(0, 4)]
                else:
                    for k in range(offspring[i][0]):
                        if random.random()<0.1:
                            offspring[i][j][k] = nb_neurons_possible[random.randint(0, 4)]
            else:
                if random.random() < 0.1 and 0.1<offspring[i][j]<0.9:
                    offspring[i][j] = offspring[i][j] + random.uniform(-0.1, 0.1)
    for j in range(len(offspring)):
        if isinstance(offspring[j][1], int):
            offspring[j][1] = [offspring[j][1]]
    return offspring

def genetic_algorithm(num_generations, num_parents_mating, sol_per_pop, num_genes, nb_neurons_possible, low_end_randomized, high_end_randomized, aim_accuracy):
    population = initialize_population(sol_per_pop, num_genes, nb_neurons_possible, low_end_randomized, high_end_randomized)
    parents, accuracies = select_parents(population, num_parents_mating)
    print('Initial best:', parents[0], '\tInitial best accuracy:', accuracies[0])
    output_accuracy=0
    while output_accuracy < aim_accuracy: 
        offspring = crossover(parents)
        offspring = mutate(offspring)
        population = parents + offspring
        parents, accuracies = select_parents(population, num_parents_mating)
        best, output_accuracy = parents[0], accuracies[0] 
        print(population)
        print('Best:', best, '\tBest Accuracy:', output_accuracy)
    
    return best, output_accuracy

num_generations = 10000
num_parents_mating = 2
sol_per_pop = 5
num_genes = 3
nb_neurons_possible = [32, 48, 64, 96, 128]
#low_end_nb_neurons = 32
#high_end_nb_neurons = 128
low_end_randomized = 0
high_end_randomized = 1

aim_accuracy = 0.98


best, output = genetic_algorithm(num_generations, num_parents_mating, sol_per_pop, num_genes, nb_neurons_possible, low_end_randomized, high_end_randomized, aim_accuracy)
print(best, output)






313/313 - 0s - loss: 0.2740 - accuracy: 0.9525 - 317ms/epoch - 1ms/step
313/313 - 0s - loss: 0.5812 - accuracy: 0.9638 - 362ms/epoch - 1ms/step
313/313 - 1s - loss: 0.3208 - accuracy: 0.9443 - 772ms/epoch - 2ms/step
313/313 - 0s - loss: 0.4998 - accuracy: 0.8469 - 316ms/epoch - 1ms/step
313/313 - 1s - loss: 0.1828 - accuracy: 0.9658 - 850ms/epoch - 3ms/step
Initial best: [1, [96], 0.05940667777126529, 0.37375061736888526] 	Initial best accuracy: 0.9657999873161316
313/313 - 1s - loss: 0.1517 - accuracy: 0.9726 - 714ms/epoch - 2ms/step
313/313 - 1s - loss: 0.5968 - accuracy: 0.9576 - 1s/epoch - 4ms/step
313/313 - 1s - loss: 0.1566 - accuracy: 0.9679 - 952ms/epoch - 3ms/step
313/313 - 1s - loss: 0.2185 - accuracy: 0.9574 - 1s/epoch - 4ms/step
313/313 - 1s - loss: 0.1928 - accuracy: 0.9614 - 1s/epoch - 4ms/step
[[1, [96], 0.05940667777126529, 0.37375061736888526], [2, [64, 128], 0.7676787150412182, 0.01663108269414737], [1, [64], 0.05940667777126529, 0.34731590993132516], [2, [96, 128]